<a href="https://colab.research.google.com/github/Dheevessh/Alzheimers-Prediction/blob/main/Alzheimers_perdiction(MRI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from PIL import Image
import io
from IPython.display import display
import random
import tensorflow as tf
import torch
import os


In [2]:
# Set a seed value
seed_value = 42

# 1. Set `PYTHONHASHSEED` environment variable
os.environ['PYTHONHASHSEED'] = str(seed_value)
# 2. Set `python` built-in pseudo-random generator
random.seed(seed_value)
# 3. Set `numpy` pseudo-random generator
np.random.seed(seed_value)
# 4. Set `tensorflow` pseudo-random generator
tf.random.set_seed(seed_value)
# 5. Set `torch` pseudo-random generator
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value) # if you are using GPU

In [ ]:
from google.colab import files
import pandas as pd

# Upload the file
uploaded = files.upload()  # select your .parquet file

# Suppose the file name is data.parquet
df = pd.read_parquet("test.parquet")
df = pd.read_parquet("train.parquet")
df.head()


In [ ]:
# Ensure pyarrow is installed in this kernel
!pip install --quiet pyarrow
!pip install pandas Pillow kagglehub ipywidgets numpy matplotlib pyarrow


In [ ]:
train_df = pd.read_parquet("train.parquet")
test_df  = pd.read_parquet("test.parquet")

def bytes_to_pixels(b: bytes) -> np.ndarray:
    """
    Convert raw image bytes (e.g. JPEG/PNG) into a 2D numpy array of pixel values (grayscale).
    """
    img = Image.open(io.BytesIO(b))  # convert to grayscale
    return np.array(img)
def extract_bytes(blob):
    """
    Unwrap a dict‐wrapped binary payload if needed,
    otherwise return blob directly.
    """
    if isinstance(blob, dict):
        # try common keys
        for key in ("bytes", "data", "image"):
            if key in blob and isinstance(blob[key], (bytes, bytearray)):
                return blob[key]
        # fallback: first bytes‐like value
        for v in blob.values():
            if isinstance(v, (bytes, bytearray)):
                return v
        raise TypeError(f"No bytes found in dict payload: {list(blob.keys())}")
    return blob

train_df["image"] = train_df["image"].apply(lambda blob: bytes_to_pixels(extract_bytes(blob)))
test_df["image"]  = test_df["image"].apply(lambda blob: bytes_to_pixels(extract_bytes(blob)))


In [ ]:
def display_images(df, n=10):
    """
    Display the first n grayscale images (numpy arrays) from df
    along with their labels. Assumes df has columns 'image' and 'label'.
    """
    for i, (_, row) in enumerate(df.head(n).iterrows(), start=1):
        pixels = row["image"]
        label = row.get("label", "")
        # Create a PIL image in 'L' mode (8-bit pixels, black and white)
        img = Image.fromarray(pixels.astype("uint8"), mode="L")
        print(f"{i}. label = {label}")
        display(img)
display_images(train_df)

In [ ]:
train_df["image"].iloc[0].shape


In [ ]:
X_train = np.stack(train_df["image"].values)
X_test  = np.stack(test_df["image"].values)

# Add channel dimension (grayscale = 1 channel)
X_train = X_train[..., np.newaxis]
X_test  = X_test[..., np.newaxis]


In [ ]:
train_df.columns


In [ ]:
train_df["label"].value_counts()


In [ ]:
print(le.classes_)


In [ ]:
train_df.head(10)
